In [39]:
# !pip install logging
# from dotenv import load_dotenv
# _ = load_dotenv()
#llama3-groq-70b-8192-tool-use-preview

import os
from langchain_groq import ChatGroq
import getpass
# KnowledgeGraphTool
import requests
from langchain_groq import ChatGroq
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langchain_core.tools import tool
import logging
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.messages import HumanMessage
from langchain_core.prompts import PromptTemplate
def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("LANGSMITH_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "LangGraph IoT Engine"
# Load a specific environment variable
GKGraph_API_KEY = os.environ.get('GKGraph_API_KEY')
TAVILY_API_KEY=os.environ.get('TAVILY_API_KEY')
# model = ChatGroq(model="llama3-8b-8192")
llm = ChatGroq(model="llama3-8b-8192" , temperature =0)

from tavily import TavilyClient
# Step 1. Instantiating your TavilyClient
tavily_client = TavilyClient(api_key=TAVILY_API_KEY)

In [40]:
from langchain_core.output_parsers import JsonOutputParser
assistant_prompt= """
Act as an assistant  and answer user queries ONLY if you can 
User queries may fall into one of the following categories:
1- Greeting/General: Greeting you or answering a general question you can ONLY answer.
2- Service Recommendation: Asking for a recommendation for a service or a place to visit, such as I want to drink coffee, I am looking for a Middle Eastern restaurant, or Do you know any close hospital?
3- Hard Question: General questions you can't answer, such as: who is the current president of the United States of America? Or any question related to current events happening as you don't have access to the current event happening right now
Your response must follow the following JSON  objects based on each category:
 1- Greeting/General: you need to provide your response ONLY in a JSON object as shown:
 {
 "query-type": "greeting-general",
 "response": write your response here
}
2-Service Recommendation: try to extract the service type and the city, country, Address, or Coordinates if mentioned. Your response is ONLY a JSON object. This JSON object must follow the following structure and set false in bool format for the value of the keys that you can't extract.
 { 
    "query-type": "greeting-general",
    "service": extracted service type, 
    "city": extracted city,
    "city": extracted country"
    "address": extracted address,
    "coordinates":[extracted  latitude, extracted longitude]
}
3- Hard Question/current events: you need to provide your response ONLY JSON object as shown:
 {
 "query-type": "hard-question"
}

here is the user query: {query}
"""

# Define your desired data structure.
class hard_question(BaseModel):
    query_type: str =Field(value="hard-question")

# class Service_Recommendation(BaseModel):
#     query_type: str =Field(value= "greeting-general")
#     service: str= Field(description="extracted service ")
#     city: str= Field(description="extracted city")
#     country: str= Field(description="extracted country)
#     address: str= Field(description="extracted address")
# Set up a parser + inject instructions into the prompt template.
parser_hard_question = JsonOutputParser(pydantic_object=hard_question)
# parser_Service_Recommendation = JsonOutputParser(pydantic_object=Joke)
                        
from langchain_core.messages import SystemMessage

system_message = SystemMessage(content=assistant_prompt)
human_message = HumanMessage(content="I want to get coffee")
messages = [system_message, human_message]


response = llm.invoke(messages)

In [41]:
from langchain_core.output_parsers import JsonOutputParser

# Initialize your LLM and parser
x=response.content
output_parser = JsonOutputParser()
# Parse the output using the parser
parsed_output = output_parser.parse(x)
parsed_output

{'query-type': 'Service Recommendation',
 'service': 'coffee',
 'city': False,
 'country': False,
 'address': False,
 'coordinates': [False, False]}